In [1]:
import pandas as pd
import yfinance as yf
#from yahoofinancials import YahooFinancials
import datetime
from pandas_datareader import data
#pip install pandas_datareader
import math

### 1. Read Stock Tickers 

In [2]:
stocks_df = pd.read_csv('quotes.csv' , encoding = 'cp1252') 
print("No. of stocks in Yahoo Finance watchlish:", len(stocks_df))
stocks_df.head()

stocks = []
for index, row in stocks_df.iterrows():
    if 'CAD' in row['Symbol']:
        continue
    if 'USD' in row['Symbol']:
        continue
    stocks.append(row['Symbol'])
    if 'AAPL' in row['Symbol']:
        print("Apple Inedx ",index)
    if 'SQ' in row['Symbol']:
        print("Square Index ", index)

print("No of Stocks", len(stocks))
print(stocks[68], stocks[129])
#tickers = [stocks[97], stocks[33]]

tickers = stocks
print(len(tickers))


No. of stocks in Yahoo Finance watchlish: 190
Apple Inedx  68
Square Index  129
No of Stocks 179
VNTR U
179


### 2. Check df1_financials file 

In [3]:
today = datetime.datetime.now()
if today.hour > 16:
    d = datetime.timedelta(days = 1) 
    a = today + d
    today_string = "stock_data\\" + str(a.date())
else:
    today_string = "stock_data\\" + str(today.date()) 
print(today_string)
filename = today_string + ".df1_financials.csv"


stock_data\2021-04-24


In [4]:
Data_exists = 0
try:
    df1_financials = pd.read_csv(filename , encoding = 'cp1252') 
    Data_exists = 1
except:
    print("No data for today exists")

### 3. Create df1_financials if required

Ref -> https://stackoverflow.com/questions/54815864/downloading-a-companies-market-cap-from-yahoo
    
marketCap
averageDailyVolume3Month
averageDailyVolume10Day
fiftyDayAverage
fiftyTwoWeekLowChangePercent
fiftyTwoWeekHighChangePercent
fiftyDayAverageChangePercent
fiftyTwoWeekRange
price

In [54]:
def size_market_cap(cap):
    if cap < 50000000:
        return("Nano")
    elif cap < 300000000:
        return("Micro")
    elif cap < 2000000000:
        return("Small")
    elif cap < 10000000000:
        return("Mid")
    elif cap <= 200000000000:
        return("Large")
    elif cap > 200000000000:
        return("Mega")
    else:
        print("Flag Error with Market Cap Sizing")
        return("Unknown")

In [63]:
n = 30
if Data_exists == 0:
    itter = math.ceil(len(tickers)/n)
    for i in range(itter):
        start_ind = i*n
        end_ind = (i+1)*n
        t = tickers[start_ind: end_ind]
        if i == 0:
            marketCap = data.get_quote_yahoo(t)['marketCap']
            price = data.get_quote_yahoo(t)['price']
        
        else:
            marketCap1 = data.get_quote_yahoo(t)['marketCap']
            marketCap = marketCap.append(marketCap1)
            price1 = data.get_quote_yahoo(t)['price']
            price = price.append(price1)
            
    sizing = []
    for cap in marketCap:
        sizing.append(size_market_cap(cap))

    numberOfShares = []
    for i in range(len(marketCap)):
        try:
            N = round(marketCap[i]/price[i])
            numberOfShares.append(N)
        except:
            numberOfShares.append(0)
    
    df1_financials= pd.DataFrame({'Ticker': tickers, 
                                  'price': price, 
                                  'marketCap': marketCap,
                                  'numberOfShares': numberOfShares, 
                                  'sizing': sizing}, index = tickers) 


    df1_financials = df1_financials[df1_financials.sizing != "Unknown"]
    print("Basic Financials extracted for ", len(df1_financials), " stocks.")
    
    df1_financials.to_csv(filename)
    Data_exists = 1
else:
    df1_financials = pd.read_csv(filename,  index_col=0,  encoding = 'cp1252') 

In [64]:
tickers = []
for index, row in df1_financials.iterrows():
    tickers.append(row['Ticker'])
print(len(df1_financials))
df1_financials.head()

174


,Ticker,price,marketCap,numberOfShares,sizing
MFC,MFC,21.30,41397186560,1943529885,Large
POW.TO,POW.TO,34.86,23458654208,672939019,Large
SFT,SFT,7.45,602072512,80815102,Small
ABNB,ABNB,174.25,101651169280,583363956,Large
WMT,WMT,139.90,398425391104,2847929886,Mega


In [15]:
# tickers = stocks

# marketCap = []
# averageDailyVolume3Month = []
# averageDailyVolume10Day = []
# fiftyDayAverage = []
# fiftyTwoWeekRange_low = []
# fiftyTwoWeekRange_high = []
# price = []

# a = data.get_quote_yahoo(tickers)['marketCap']
# for value in a:
#     marketCap.append(float(value))
    
# a = data.get_quote_yahoo(tickers)['averageDailyVolume3Month']
# for value in a:
#     averageDailyVolume3Month.append(int(value))
    
# a = data.get_quote_yahoo(tickers)['averageDailyVolume10Day']
# for value in a:
#     averageDailyVolume10Day.append(int(value))
    
# a = data.get_quote_yahoo(tickers)['fiftyDayAverage']
# for value in a:
#     fiftyDayAverage.append(float(value))

# a = data.get_quote_yahoo(tickers)['fiftyTwoWeekRange']
# for value in a:
#     low, high = str(value).split("-")
#     fiftyTwoWeekRange_low.append(float(low))
#     fiftyTwoWeekRange_high.append(float(high))

# a = data.get_quote_yahoo(tickers)['price']
# for value in a:
#     price.append(float(value))



### 3. Get 420+ Day Raw Data

In [40]:
def days(Day_of_year, Year):
    days = (Year - 2020)*365 + Day_of_year
    days = days + int(2020 - Year)
    return(int(days))

In [41]:
def pull_60d_stock_data(ticker):
    #gets the data of the stock for the last 40 months
    
    #Get the dates
    today = datetime.datetime.now()
    end_date = str(today.date())

    d = datetime.timedelta(days = 420) 
    a = today - d
    start_date = str(a.date())
    #print(start_date, end_date, type(start_date))
    
    df_stock = yf.download(ticker, 
                      start= start_date, 
                      interval='60m',
                      progress=False)  
    return(df_stock)

In [42]:
def clean_60d_stock_data(df_stock, ticker):
    Ticker = []
    Datetime = []
    #Year = []
    #Day_of_week = []
    #Day_of_year = []
    Day = []
    Sessions = []
    Open = []
    High = []
    Low = []
    Close = []
    Volume = []
    Dollar_Interest = []
    key=[]
    
    for index, row in df_stock.iterrows(): 
        #Year.append(2021)
        Ticker.append(ticker)
        Open.append(row['Open'])
        High.append(row['High'])
        Low.append(row['Low'])
        Close.append(row['Close'])
        Volume.append(row['Volume'])
        Datetime.append(index)
        key.append(ticker+str(index))
        
    df_stock_clean = pd.DataFrame({'Ticker': ticker, 
                                   'Datetime': Datetime})
                                   #,'Year' : Year 
    df_stock_clean['Year'] = df_stock_clean['Datetime'].dt.year
    df_stock_clean['Day_of_year'] = df_stock_clean['Datetime'].dt.dayofyear
    df_stock_clean['Day_of_week'] = df_stock_clean['Datetime'].dt.dayofweek
    
    Session = 0
    d_o_w = df_stock_clean['Day_of_week'][0]
    for index, row in df_stock_clean.iterrows(): 
        Day.append(days(row['Day_of_year'], row['Year']))
        
        if row['Day_of_week'] != d_o_w:
            Session = 0
            d_o_w = row['Day_of_week']
        else:
            Session = Session + 1
        Sessions.append(Session)
            
    df_stock_clean['Day'] = Day
    df_stock_clean['Sessions'] = Sessions
    df_stock_clean['Open'] = Open
    df_stock_clean['High'] = High
    df_stock_clean['Low'] = Low
    df_stock_clean['Close'] = Close
    df_stock_clean['Volume'] = Volume
    
    
    for index, row in df_stock_clean.iterrows(): 
        Dollar_Interest.append(row['Volume']*row['Close']/1000000)
     
    df_stock_clean['Dollar_Interest'] = Dollar_Interest 
    df_stock_clean['key'] = key
    return df_stock_clean

In [43]:
ticker = "SQ"
df_stock = pull_60d_stock_data(ticker)
df_stock_clean = clean_60d_stock_data(df_stock, ticker)
df_stock_clean.head()

,Ticker,Datetime,Year,Day_of_year,Day_of_week,Day,Sessions,Open,High,Low,Close,Volume,Dollar_Interest,key
0,SQ,2020-03-02 09:30:00-05:00,2020,62,0,62,1,82.800003,84.419998,80.150002,80.349998,4117183.0,330.815648,SQ2020-03-02 09:30:00-05:00
1,SQ,2020-03-02 10:30:00-05:00,2020,62,0,62,2,80.410004,82.300003,80.029999,81.830002,2840954.0,232.475271,SQ2020-03-02 10:30:00-05:00
2,SQ,2020-03-02 11:30:00-05:00,2020,62,0,62,3,81.860001,82.220001,80.769997,81.529999,2104506.0,171.580372,SQ2020-03-02 11:30:00-05:00
3,SQ,2020-03-02 12:30:00-05:00,2020,62,0,62,4,81.489998,82.000000,80.830002,80.900002,1419964.0,114.875090,SQ2020-03-02 12:30:00-05:00
4,SQ,2020-03-02 13:30:00-05:00,2020,62,0,62,5,80.949997,81.519997,80.629997,80.944603,1153768.0,93.391293,SQ2020-03-02 13:30:00-05:00


In [44]:
ticker = tickers[0]
df_stock = pull_60d_stock_data(ticker)
df_stock_clean = clean_60d_stock_data(df_stock, ticker)

for ticker in tickers:
    if ticker == tickers[0]:
        continue
    
    df_stock = pull_60d_stock_data(ticker)
    df_stock_clean1 = clean_60d_stock_data(df_stock, ticker)
    
    df_stock_clean = pd.concat([df_stock_clean, df_stock_clean1])
    
    #Ref -> https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
    df_stock_clean = df_stock_clean.drop_duplicates(subset =['key'])
    
    df_stock_clean = df_stock_clean.reset_index(drop=True)
    
 

,Ticker,Datetime,Year,Day_of_year,Day_of_week,Day,Sessions,Open,High,Low,Close,Volume,Dollar_Interest,key
0,MFC,2020-03-02 09:30:00-05:00,2020,62,0,62,1,16.879999,16.879999,16.600000,16.730000,1710327.0,28.613770,MFC2020-03-02 09:30:00-05:00
1,MFC,2020-03-02 10:30:00-05:00,2020,62,0,62,2,16.745001,17.030001,16.650000,16.990000,820795.0,13.945307,MFC2020-03-02 10:30:00-05:00
2,MFC,2020-03-02 11:30:00-05:00,2020,62,0,62,3,17.000000,17.195000,16.969999,17.135000,837242.0,14.346142,MFC2020-03-02 11:30:00-05:00
3,MFC,2020-03-02 12:30:00-05:00,2020,62,0,62,4,17.139999,17.190001,17.025000,17.045000,577529.0,9.843982,MFC2020-03-02 12:30:00-05:00
4,MFC,2020-03-02 13:30:00-05:00,2020,62,0,62,5,17.049999,17.100000,17.014999,17.049999,497095.0,8.475469,MFC2020-03-02 13:30:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317308,CTRM,2021-04-23 11:30:00-04:00,2021,113,4,477,2,0.487149,0.497499,0.485000,0.487799,15949571.0,7.780185,CTRM2021-04-23 11:30:00-04:00
317309,CTRM,2021-04-23 12:30:00-04:00,2021,113,4,477,3,0.487850,0.494800,0.487300,0.492900,6329001.0,3.119565,CTRM2021-04-23 12:30:00-04:00
317310,CTRM,2021-04-23 13:30:00-04:00,2021,113,4,477,4,0.492996,0.498000,0.490000,0.490300,7281572.0,3.570155,CTRM2021-04-23 13:30:00-04:00
317311,CTRM,2021-04-23 14:30:00-04:00,2021,113,4,477,5,0.490500,0.491699,0.487901,0.488000,6146431.0,2.999458,CTRM2021-04-23 14:30:00-04:00


In [71]:
df_stock_clean = df_stock_clean[df_stock_clean.Volume != 0] 
df_stock_clean = df_stock_clean.reset_index(drop=True)
print(len(df_stock_clean))
df_stock_clean.head()

309379


,Ticker,Datetime,Year,Day_of_year,Day_of_week,Day,Sessions,Open,High,Low,Close,Volume,Dollar_Interest,key
0,MFC,2020-03-02 09:30:00-05:00,2020,62,0,62,1,16.879999,16.879999,16.600000,16.730000,1710327.0,28.613770,MFC2020-03-02 09:30:00-05:00
1,MFC,2020-03-02 10:30:00-05:00,2020,62,0,62,2,16.745001,17.030001,16.650000,16.990000,820795.0,13.945307,MFC2020-03-02 10:30:00-05:00
2,MFC,2020-03-02 11:30:00-05:00,2020,62,0,62,3,17.000000,17.195000,16.969999,17.135000,837242.0,14.346142,MFC2020-03-02 11:30:00-05:00
3,MFC,2020-03-02 12:30:00-05:00,2020,62,0,62,4,17.139999,17.190001,17.025000,17.045000,577529.0,9.843982,MFC2020-03-02 12:30:00-05:00
4,MFC,2020-03-02 13:30:00-05:00,2020,62,0,62,5,17.049999,17.100000,17.014999,17.049999,497095.0,8.475469,MFC2020-03-02 13:30:00-05:00


In [69]:
len(df_stock_clean)

317313

In [70]:
filename2 = today_string + ".df1_420d_1hr_raw_data.csv"
df_stock_clean.to_csv(filename2)

### 4. Prepare Data (for Classification Models)

In [47]:
df_stock_clean[100:].head(35)
df_stock_clean['Close'].mean()

135.96001969604686

In [67]:
df_stock_clean['Volume'].mean()

1892528.0086003952

In [48]:
df_stock_clean['Close'].max()

3532.393310546875

In [49]:
df_stock_clean['Close'][2183]

21.290000915527344

In [65]:
df1_financials['marketCap']["MFC"]

41397186560

In [81]:
uid_n = [] # ticker_day_session
ticker_n = []
market_cap_n = []

price_close_n = []
price_close_10day_100_n = []
price_close_10day_z_n = []
price_close_yesterday_close_100_n = []
price_close_yesterday_z_n = []

Volume_n = []
Volume_10day_100_n = []
Volume_10day_z_n = []
Volume_yesterday_z_n = []

Dollar_Interest_n = []
Dollar_Interest_10day_100_n = []
Dollar_Interest_10day_z_n = []
Dollar_Interest_yesterday_z_n = []

Day_of_week_n = []
Sessions_n = []

percent_gain_10_today = []
percent_gain_5_today = []
percent_gain_10_tomorrow = []
percent_gain_5_tomorrow = []

percent_loss_10_today = []
percent_loss_5_today = []
percent_loss_10_tomorrow = []
percent_loss_5_tomorrow = []

next_session_gain = []
next_session_loss = []

for ticker in tickers:
    print(ticker)
#     if "AZF" not in ticker:
#         continue
    df_stock_tick = df_stock_clean[df_stock_clean.Ticker == ticker]
    df_stock_tick = df_stock_tick.reset_index(drop=True)
    
    min_days = df_stock_tick['Day'].min()
    max_days = df_stock_tick['Day'].max()
    for index, row in df_stock_tick.iterrows():
        
        uid = row['Ticker'] + "_" + str(row["Day"]) + "_" + str(row["Sessions"])
        if row['Day'] < min_days + 14:
            continue
            
        if row['Day'] == max_days:
            continue
            
        #dataframe for stocks 10 days before
        df_stock_tick_10day = df_stock_tick[df_stock_tick.Day >= row['Day']-14] 
        df_stock_tick_10day = df_stock_tick_10day[df_stock_tick_10day.Day < row['Day']]
        df_stock_tick_10day = df_stock_tick_10day[df_stock_tick_10day.Sessions < row['Sessions']]
        
        #dataframe for stocks previous day(yesterday)
        if df_stock_clean['Day'].isin([row['Day']-1]).any().any():
            yesterday = 1
        elif df_stock_clean['Day'].isin([row['Day']-2]).any().any():
            yesterday = 2
        elif df_stock_clean['Day'].isin([row['Day']-3]).any().any(): 
            yesterday = 3
        elif df_stock_clean['Day'].isin([row['Day']-4]).any().any(): 
            yesterday = 4
        else:
            print("Yesterday not found for", row['Day'])
            yesterday = 1
        df_stock_tick_yesterday = df_stock_tick[df_stock_tick.Day == row['Day']-yesterday]
        max_session = df_stock_tick_yesterday['Sessions'].max()
        df_stock_tick_yesterday_last_sess = df_stock_tick_yesterday[df_stock_tick_yesterday.Sessions == max_session]
        yesterday_close_price = df_stock_tick_yesterday_last_sess['Close'].max()
        
        
        ############ Variables ############
        #Basic Variables
        uid_n.append(uid)
        ticker_n.append(ticker)
        
        market_cap = row['Close'] * df1_financials['marketCap'][ticker]
        market_cap_n.append(market_cap)
        
        #Price Variables
        price_close_n.append(row['Close'])
        
        mean_price_10_day = df_stock_tick_10day['Close'].mean()
        price_close_10day_100 = (row['Close']*100)/mean_price_10_day
        price_close_10day_100_n.append(price_close_10day_100)
        
        st_dev_price_10_day = df_stock_tick_10day['Close'].std()
        
        if st_dev_price_10_day == 0:
            price_close_10day_z = 2
        else:
            price_close_10day_z = (row['Close'] - mean_price_10_day)/st_dev_price_10_day
        
        
        price_close_10day_z_n.append(price_close_10day_z)
        
        price_close_yesterday_close_100 = (row['Close']*100)/yesterday_close_price
        price_close_yesterday_close_100_n.append(price_close_yesterday_close_100)
        
        mean_price_yesterday = df_stock_tick_yesterday['Close'].mean()
        st_dev_price_yesterday = df_stock_tick_yesterday['Close'].std()
        if st_dev_price_yesterday == 0:
            price_close_yesterday_z = 2
        else:
            price_close_yesterday_z = (row['Close'] - mean_price_yesterday)/st_dev_price_yesterday
        price_close_yesterday_z_n.append(price_close_yesterday_z)
        
        #Volume Variables
        Volume_n.append(row['Volume'])
        
        mean_Volume_10_day = df_stock_tick_10day['Volume'].mean()
        try:
            Volume_10day_100 = (row['Volume']*100)/mean_Volume_10_day
        except:
            print(uid)
            break
        Volume_10day_100_n.append(Volume_10day_100)
        
        st_dev_Volume_10_day = df_stock_tick_10day['Volume'].std()
        Volume_10day_z = (row['Volume'] - mean_Volume_10_day)/st_dev_Volume_10_day
        Volume_10day_z_n.append(Volume_10day_z)
        
        mean_Volume_yesterday = df_stock_tick_yesterday['Volume'].mean()
        st_dev_Volume_yesterday = df_stock_tick_yesterday['Volume'].std()
        Volume_yesterday_z = (row['Volume'] - mean_price_yesterday)/st_dev_Volume_yesterday
        Volume_yesterday_z_n.append(Volume_yesterday_z)
        
        #Dollar Interest Variables
        Dollar_Interest_n.append(row['Dollar_Interest'])
        
        mean_Dollar_Interest_10_day = df_stock_tick_10day['Dollar_Interest'].mean()
        Dollar_Interest_10day_100 = (row['Dollar_Interest']*100)/mean_Dollar_Interest_10_day
        Dollar_Interest_10day_100_n.append(Dollar_Interest_10day_100)
        
        st_dev_Dollar_Interest_10_day = df_stock_tick_10day['Dollar_Interest'].std()
        Dollar_Interest_10day_z = (row['Dollar_Interest'] - mean_Dollar_Interest_10_day)/st_dev_Dollar_Interest_10_day
        Dollar_Interest_10day_z_n.append(Dollar_Interest_10day_z)
        
        mean_Dollar_Interest_yesterday = df_stock_tick_yesterday['Dollar_Interest'].mean()
        st_dev_Dollar_Interest_yesterday = df_stock_tick_yesterday['Dollar_Interest'].std()
        Dollar_Interest_yesterday_z = (row['Dollar_Interest'] - mean_price_yesterday)/st_dev_Dollar_Interest_yesterday
        Dollar_Interest_yesterday_z_n.append(Dollar_Interest_yesterday_z)
        
        #Other Time Variables
        Day_of_week_n.append(row['Day_of_week'])
        Sessions_n.append(row['Sessions'])
        
        #         #dataframe for stocks same day in the future
#         df_stock_tick_day = df_stock_tick[df_stock_tick.Day == row['Day']]
#         df_stock_tick_day = df_stock_tick_day[df_stock_tick_yesterday.Sessions > row['Sessions']]
        
        #dataframe for stocks next day(tommorow)
        #dataframe for stocks previous day(yesterday)
        if df_stock_clean['Day'].isin([row['Day']+1]).any().any():
            tomorrow = 1
        elif df_stock_clean['Day'].isin([row['Day']+2]).any().any():
            tomorrow = 2
        elif df_stock_clean['Day'].isin([row['Day']+3]).any().any(): 
            tomorrow = 3
        elif df_stock_clean['Day'].isin([row['Day']+4]).any().any(): 
            tomorrow = 4
        else:
            print("Tomorrow not found for", row['Day'])
            tomorrow = 1
        df_stock_tick_tomorrow = df_stock_tick[df_stock_tick.Day == row['Day']+ tomorrow]
        
        tomorrow_max = df_stock_tick_tomorrow['High'].max()
        gain_tom = (tomorrow_max - row['Close'])/row['Close'] *100
        if gain_tom > 5:
            percent_gain_5_tomorrow.append(1)
        else:
            percent_gain_5_tomorrow.append(0)
        if gain_tom > 10:
            percent_gain_10_tomorrow.append(1)
        else:
            percent_gain_10_tomorrow.append(0)
            
        tomorrow_min = df_stock_tick_tomorrow['Low'].min()
        loss_tom = (row['Close'] - tomorrow_min)/row['Close'] *100
        if loss_tom > 5:
            percent_loss_5_tomorrow.append(1)
        else:
            percent_loss_5_tomorrow.append(0)
        if loss_tom > 10:
            percent_loss_10_tomorrow.append(1)
        else:
            percent_loss_10_tomorrow.append(0)
        
        df_stock_tick_today_future = df_stock_tick[df_stock_tick.Day == row['Day']]
        df_stock_tick_today_future = df_stock_tick_tomorrow[df_stock_tick_tomorrow.Sessions > row['Sessions']]
        today_max = df_stock_tick_tomorrow['Close'].max()
        
        today_max = df_stock_tick_today_future['High'].max()
        gain_today = (today_max - row['Close'])/row['Close'] *100
        if gain_today > 5:
            percent_gain_5_today.append(1)
        else:
            percent_gain_5_today.append(0)
        if gain_today > 10:
            percent_gain_10_today.append(1)
        else:
            percent_gain_10_today.append(0)
            
        today_min = df_stock_tick_today_future['Low'].min()
        loss_today = (row['Close'] - today_min)/row['Close'] *100
        if loss_today > 5:
            percent_loss_5_today.append(1)
        else:
            percent_loss_5_today.append(0)
        if loss_tom > 10:
            percent_loss_10_today.append(1)
        else:
            percent_loss_10_today.append(0)
        
        next_session_change = (df_stock_clean['Close'][index+1] - row['Close'])/row['Close'] *100
        if next_session_change > 1.5:
            next_session_gain.append(1)
            next_session_loss.append(0)
        elif next_session_change < -1.5:
            next_session_gain.append(0)
            next_session_loss.append(1)
        else:
            next_session_gain.append(0)
            next_session_loss.append(0)

MFC
POW.TO
SFT
ABNB
WMT
AMD
SNPS
TCEHY
TWTR
NTLA
TXG
NVTA
CRSP
EXAS
TWLO
BIDU
SPOT
ZM
ROKU
DOCU
UI
W
T
TGT
SPG
JPM
RDFN
GOOGL
GOOG
Z
GBTC
MARA
HIVE.V
COIN
GLXY.TO
BTBT
AMC
AZFL
DNN
VPER
BB
GM
TDC
TSLA
MGM
EXPR
KMPH
CGX.TO
WBA
FRPT
AQN.TO
STLC.TO
LXXGF
NEPT.TO
FB
LAZR
DAL
FORD
BFLY
MSFT
AC.TO
BA
AMZN
AAPL
PLTR
BEP
VNTR
BBY
NKE
ADBE
PG
CHKP
BRK-B
REGN
COST
SPY
ADSK
AAL
LOGI
MFI.TO
NOK
CSV
TSIA
ABX.TO
PYPL
PANW
VZ
QS
NVDA
GIS
V
MVIS
AKAM
AES
BAC
SAII
MO
CRM
WORK
BCE.TO
SJR-B.TO
JNJ
UPS
BNS.TO
CWB.TO
CU.TO
GMHIU
BABA
NFLX
AZN
ILMN
T.TO
PFE
CCA.TO
SQ
CLSK
DIS
GSAH
DDD
RCI-B.TO
BYND
BLX
SNAP
FDX
U
BIOL
TNXP
DND.TO
AACQ
NIO
MAC
NGVT
TDOC
SHOP
GS
CVX
NCNO
XOM
SNP
OCGN
HTH
MRNA
FSLY
KXS.TO
PTON
XPEV
CCIV
SNDL
UPWK
NSPR
ODT
IPOE
INO
NAKD
LMND
MACK
WLLW.TO
FVRR
GME
CHCI
NVAX
VERO
DDOG
RAAS
ETSY
CHPT
FLNT
TSNP
ENPH
GEVO
EBON
NCTY
CAN
CTRM


In [82]:
print(len(uid_n))
print(len(price_close_yesterday_z_n))
print(len(Day_of_week_n))
print(len(percent_gain_10_today))
print(len(percent_gain_5_tomorrow))
print(len(percent_loss_10_tomorrow))
print(len(next_session_gain))

296871
296871
296871
296871
296871
296871
296871


In [83]:
df_stock_clean_rf = pd.DataFrame({'uid': uid_n,
                               'ticker': ticker_n,
                               'market_cap': market_cap_n,
                               
                               'price_close': price_close_n,
                               'price_close_10day_100': price_close_10day_100_n,
                               'price_close_10day_z': price_close_10day_z_n,
                               'price_close_yesterday_close_100': price_close_yesterday_close_100_n, 
                               'price_close_yesterday_z': price_close_yesterday_z,
                               
                               'Volume': Volume_n,
                               'Volume_10day_100': Volume_10day_100_n,
                               'Volume_10day_z': Volume_10day_z_n, 
                               'Volume_yesterday_z': Volume_yesterday_z_n,
                               
                               'Dollar_Interest' : Dollar_Interest_n,
                               'Dollar_Interest_10day_100': Dollar_Interest_10day_100_n,
                               'Dollar_Interest_10day_z' : Dollar_Interest_10day_z_n,
                               'Dollar_Interest_yesterday_z': Dollar_Interest_yesterday_z_n,
                               
                               'Day_of_week' : Day_of_week_n,
                               'Sessions' : Sessions_n,
                               
                               'percent_gain_10_today': percent_gain_10_today,
                               'percent_gain_5_today' : percent_gain_5_today,
                               'percent_gain_10_tomorrow' : percent_gain_10_tomorrow,
                               'percent_gain_5_tomorrow' : percent_gain_5_tomorrow,
                               
                               'percent_loss_10_today' : percent_loss_10_today,
                               'percent_loss_5_today' : percent_loss_5_today,
                               'percent_loss_10_tomorrow' : percent_loss_10_tomorrow,
                               'percent_loss_5_tomorrow' : percent_loss_5_tomorrow,
                               
                               'next_session_gain' : next_session_gain,
                               'next_session_loss' : next_session_loss})

In [84]:
filename3 = today_string + ".df_stock_clean_rf.csv"
df_stock_clean_rf.to_csv(filename3)

Company constants (Calculated at the close of the prebious day)

    c_market_cap        - Market Capital in $ 
    
    c_market_cap_vol    - Market Capital in volumes - volume * price (if price is low, volume will be higher)
    
    c_price             - Price
    
    Other things to consider before buying a stock
    
    c_variance_price    - Volatility in Price
    
                              - find the mean, deviation, sq deviations, mean of sqares(Variance)
                              
    c_variance_volume   - Volatility in Volume 
  
  
Trading Variables (Useful variables for Time Series Models) 

    v_volume_price      - Money traded in a session - (Volume * price)
    
    v_volume            - Volume traded in a session (scaled to 100)
    
    v_price             - Price (scaled to 100)
    
    
    v_rsi               - RSI (0 to 100)
    
Row Variables Variables ((Useful variables for Random Forest or XG Boost Models)

    v_volume_price_rel_ps      - Money traded in a session - (Volume * price)
    
    v_volume_price_rel_pd      - Money traded in a session - (Volume * price)
    
    v_volume_rel_ps            - Volume traded in a session (scaled to 100)
    
    v_volume_rel_pd            - Volume traded in a session (scaled to 100)
    
    v_price_rel_ps             - Price (scaled to 100)
    
    v_price_rel_pd             - Price (scaled to 100)
    
    v_rsi                      - RSI (0 to 100)
    
    *ps = previous session
    
    *pd = previous day
    
    
    
Row 

Ref -> https://towardsdatascience.com/python-how-to-get-live-market-data-less-than-0-1-second-lag-c85ee280ed93